In [ ]:
"""
Result: 0.79728

Logistic

Missing Value Imputation Strategies:
spending columns → 0
age -> median
categorical → 'Unknown'

New Features:
None

Deleted Features:
Name
"""

"\nLogistic\n\nMissing Value Imputation Strategies:\nspending columns → 0\nage -> median\ncategorical → 'Unknown'\n\nNew Features:\nNone\n\nDeleted Features:\nName\n"

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
import missingno as msno

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

y = train["Transported"].astype(int)

X_train = train.drop(columns=["Transported"])
X_test = test.copy()

In [ ]:
# spending columns → 0
spend_cols = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
X_train[spend_cols] = X_train[spend_cols].fillna(0)
X_test[spend_cols] = X_test[spend_cols].fillna(0)

# Age → median
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].median())
X_test['Age'] = X_test['Age'].fillna(X_train['Age'].median())

# categorical → 'Unknown'
cat_cols = ['HomePlanet','CryoSleep','Destination','Cabin','VIP']
for c in cat_cols:
    X_train[c] = X_train[c].fillna('Unknown')
    X_test[c] = X_test[c].fillna('Unknown')


In [ ]:
drop_cols = ['Name', 'Cabin']
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [ ]:
pid_test = test["PassengerId"]       
X_train = X_train.drop(columns=["PassengerId"])
X_test  = X_test.drop(columns=["PassengerId"])

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# train / test 컬럼 정렬
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gb = GradientBoostingClassifier(
    n_estimators=200,     
    learning_rate=0.05,  
    max_depth=3,       
    subsample=0.8,    
    random_state=42
)

gb.fit(X_train, y_train)

val_pred = gb.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, val_pred))


Validation Accuracy: 0.7832087406555491


In [19]:
test_pred = gb.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": pid_test,
    "Transported": test_pred.astype(bool)
})

submission.to_csv("./submission/v1_gb.csv", index=False)


In [20]:
feat_imp = (
    pd.Series(gb.feature_importances_, index=X_train.columns)
      .sort_values(ascending=False)
)

print(feat_imp.head(15))


CryoSleep_True             0.230731
Spa                        0.139319
CryoSleep_False            0.133579
VRDeck                     0.114074
RoomService                0.102840
FoodCourt                  0.091043
HomePlanet_Earth           0.065674
ShoppingMall               0.048331
Age                        0.036307
HomePlanet_Europa          0.022710
Destination_55 Cancri e    0.005400
Destination_TRAPPIST-1e    0.003171
HomePlanet_Mars            0.002255
HomePlanet_Unknown         0.002026
VIP_Unknown                0.000666
dtype: float64
